In [1]:
import pandas as pd
import seaborn as sns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# Designate the file path, and various sets of features of interest.
filepath = "../data/elephant.csv"
features = ["country", "bin_year", "pop", "RRinc"]
features_ypi = ["bin_year", "pop", "RRinc"]
features_country = ["country", "bin_year", "pop", "RRinc"]
features_year_quintile = ["bin_year", "quintile", "RRinc"]
features_year_region = ["bin_year", "region", "quintile", "RRinc"]

# Read the data and reset the index.
elephant = pd.read_csv(filepath, header=0)
df = elephant.reset_index(drop=True)

# Extract features and observations of interest
df = df[(df.mysample == 1) & (df["bin_year"].isin([1988,2008]))][features_ypi]
print("="*40)
print("\nRaw Study Data")
print(df.info())
print("-"*40)
print(df.head())
print("-"*40)
print(df.tail())
print("="*40)



Raw Study Data
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1960 entries, 40 to 5779
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   bin_year  1960 non-null   int64  
 1   pop       1960 non-null   float64
 2   RRinc     1959 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 61.2 KB
None
----------------------------------------
    bin_year      pop   RRinc
40      2008  0.31814   736.0
41      2008  0.31814   962.0
42      2008  0.31814  1162.0
43      2008  0.31814  1352.0
44      2008  0.31814  1548.0
----------------------------------------
      bin_year       pop   RRinc
5775      2008  1.237961   320.0
5776      2008  1.237961   415.0
5777      2008  1.237961   560.0
5778      2008  1.237961   828.0
5779      2008  1.237961  2195.0


In [3]:
# Obtain basic descriptive statistics of income by year.
dfg = df.groupby(by=["bin_year"])
print("\n")
print("="*80)
print("           Income statistics grouped by year")
print("-"*80)
print(dfg["RRinc"].describe())
print("-"*80)
print("          Population statistics grouped by year")
print("-"*80)
print(dfg["pop"].describe())
print("="*80)





           Income statistics grouped by year
--------------------------------------------------------------------------------
           count         mean          std   min    25%     50%      75%      max
bin_year                                                                         
1988       750.0  4594.340000  5504.011711  82.0  787.5  2644.0  6217.25  43279.0
2008      1209.0  5130.076923  7721.997274  38.0  720.0  1994.0  6244.00  72925.0
--------------------------------------------------------------------------------
          Population statistics grouped by year
--------------------------------------------------------------------------------
           count      mean        std       min       25%       50%       75%        max
bin_year                                                                                
1988       750.0  5.526245  12.414399  0.073653  0.512952  1.476009  5.443372  81.917210
2008      1210.0  5.042867  12.127811  0.008696  0.476821  1.123709

In [4]:
# Process the data: Sort data by income, create cumulative sum of population and create quintile by cumulative population grouped by year.
dfg = df.groupby(by=["bin_year"]).apply(lambda x: (x.sort_values("RRinc")
                                                    .assign(cumpop=lambda df: df["pop"].cumsum()))\
                                                    .assign(quintile=lambda df: pd.qcut(df["cumpop"], 20, labels=range(1,21))))\
                                                        .drop(columns=["bin_year", "pop", "cumpop"])\
                                                        .groupby(by=["bin_year","quintile"]).mean()
print("="*80)
print("         Data in Population Quintiles, Sorted by Income")      
print(dfg)                                                                                              
print("-"*80)


         Data in Population Quintiles, Sorted by Income
                          RRinc
bin_year quintile              
1988     1           184.315789
         2           306.810811
         3           414.447368
         4           534.702703
         5           689.236842
         6           910.189189
         7          1177.684211
         8          1524.729730
         9          1903.736842
         10         2399.972973
         11         2918.648649
         12         3498.000000
         13         4157.459459
         14         4926.394737
         15         5793.810811
         16         6898.078947
         17         8235.945946
         18        10069.526316
         19        13304.567568
         20        21884.421053
2008     1           158.245902
         2           293.433333
         3           400.163934
         4           510.033333
         5           643.426230
         6           793.233333
         7           962.344262
         8      

In [5]:
# Pivot the data
dfp = dfg.reset_index().pivot(index="quintile", columns="bin_year", values="RRinc").reset_index()
dfp["pct_growth"] = (dfp[2008]-dfp[1988])/dfp[1988]*100
print("="*40)
print("\nPivot Data")
print("-"*40)
print(dfp)
print("="*40)


Pivot Data
----------------------------------------
bin_year quintile          1988          2008  pct_growth
0               1    184.315789    158.245902  -14.144142
1               2    306.810811    293.433333   -4.360171
2               3    414.447368    400.163934   -3.446381
3               4    534.702703    510.033333   -4.613661
4               5    689.236842    643.426230   -6.646570
5               6    910.189189    793.233333  -12.849620
6               7   1177.684211    962.344262  -18.285033
7               8   1524.729730   1180.466667  -22.578629
8               9   1903.736842   1462.459016  -23.179560
9              10   2399.972973   1792.200000  -25.324159
10             11   2918.648649   2194.933333  -24.796247
11             12   3498.000000   2731.049180  -21.925409
12             13   4157.459459   3379.650000  -18.708768
13             14   4926.394737   4318.344262  -12.342707
14             15   5793.810811   5501.133333   -5.051554
15             16  